In [1]:
import os

In [2]:
%pwd #to check directory location

'd:\\Codes\\ChickenDiseaseClassification\\research'

In [3]:
#we have to change directory to project folder
os.chdir("../")

In [4]:
%pwd

'd:\\Codes\\ChickenDiseaseClassification'

In [5]:
import sys
sys.path.append('D:\Codes\ChickenDiseaseClassification')

In [6]:
sys.path

['c:\\Users\\User\\anaconda3\\envs\\venv\\python38.zip',
 'c:\\Users\\User\\anaconda3\\envs\\venv\\DLLs',
 'c:\\Users\\User\\anaconda3\\envs\\venv\\lib',
 'c:\\Users\\User\\anaconda3\\envs\\venv',
 '',
 'c:\\Users\\User\\anaconda3\\envs\\venv\\lib\\site-packages',
 'c:\\Users\\User\\anaconda3\\envs\\venv\\lib\\site-packages\\cnnclassifier-0.0.0-py3.8.egg',
 'd:\\codes\\chickendiseaseclassification\\src',
 'c:\\Users\\User\\anaconda3\\envs\\venv\\lib\\site-packages\\win32',
 'c:\\Users\\User\\anaconda3\\envs\\venv\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\User\\anaconda3\\envs\\venv\\lib\\site-packages\\Pythonwin',
 'D:\\Codes\\ChickenDiseaseClassification']

In [7]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
from dataclasses import dataclass
from pathlib import Path

#this is same as config.yaml
@dataclass(frozen=True)
class DataIngestionConfig: #entity name
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [9]:
#5. create and update configmanager for config->configuration.py
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
#6. update the components
import os
import urllib.request as request
import zipfile
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    #download the data with url
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    #extract the downloaded zip file
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
#7. Create the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-02 15:30:51,653: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-02 15:30:51,656: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-02 15:30:51,657: INFO: common: created directory at: artifacts]
[2024-08-02 15:30:51,659: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-02 15:31:00,456: INFO: 1645260119: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4A34:1F942E:346746:3D3BB5:66ACAEC2
Accept-Ranges: bytes
Date: Fri, 02 Aug 2024 10:02:46 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4729-BOM
X-Cache: M